In [ ]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date, datetime
import os, sys 
import io
import ee



In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 768)

In [ ]:
#mounting the google drive to access the files
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

phys_csv_path = "/content/drive/My Drive/cms_19/physicians.csv"

In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
# df = pd.read_csv(csv_path, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, }, parse_dates=[10] )
pdf = pd.read_csv(phys_csv_path, index_col=None, header='infer')

pdf.sample(30)

In [ ]:
len(pdf)

In [ ]:
pdf.columns.to_list()

In [ ]:
own_csv_path="/content/drive/My Drive/cms_19/ownership.csv"

In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
# df = pd.read_csv(csv_path, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, }, parse_dates=[10] )
own_df = pd.read_csv(own_csv_path, index_col=None, header='infer')



In [ ]:
own_df.sample(50)

In [ ]:
own_df.columns.to_list()

In [ ]:
own_df.Change_Type.value_counts()

In [ ]:
counter=0
count_of_new=0
count_of_not_new=0
for i in own_df.Change_Type:
  
  if i == 'NEW':
    counter=counter+1
    
    count_of_new= count_of_new+1
  if i != 'NEW':
      counter=counter+1
      count_of_not_new= count_of_not_new+1
      
print(f'this is the count of not new {counter}')
print(f'this is the count of not new {count_of_new}')
print(f'this is the count of not new {count_of_not_new}')


     

    

In [ ]:
res_csv_path="/content/drive/My Drive/cms_19/research_payments.csv"

In [ ]:
res_df = pd.read_csv(res_csv_path, index_col=None, header='infer')


In [ ]:
res_df.sample(5)

In [ ]:
res_df.columns.to_list()

In [ ]:
gen_csv_path="/content/drive/My Drive/cms_19/research_payments.csv"

In [ ]:
gen_pay_df = pd.read_csv(gen_csv_path, index_col=None, header='infer')


In [ ]:
gen_pay_df.sample(5)

In [ ]:
gen_pay_df.columns.to_list()